In [ ]:
import torch
import numpy as np

# Dummy gradient array of shape (height, width, 2)
# where grad[:, :, 0] would represent the x-component of the gradient
# and grad[:, :, 1] would represent the y-component
grad = torch.tensor([[[1.0, 2.0], [2.0, 1.0]],
                     [[-1.0, -2.0], [-2.0, -1.0]]])

# Creating Filters
filters = torch.arange(9).reshape(3, 3)[None] == torch.arange(9)[:, None, None]
print("Filters Before Reordering:\n", filters.float())

# Reordering Filters
filters = torch.stack([filters[i] for i in [5, 8, 7, 6, 3, 0, 1, 2]]).float()
print("Filters After Reordering:\n", filters)

# Gradient Angles
gdangle = torch.atan2(-grad[:, :, 1], -grad[:, :, 0])
print("Gradient Angles (radians):\n", gdangle)

# Binned Angles
binned = (gdangle * 180 / np.pi % 360) / 45
print("Binned Angles:\n", binned)

# Probability Calculation
prob = torch.clamp(1 - torch.abs(binned - torch.arange(9)[:, None, None]), min=0)
print("Probabilities Before Adjustment:\n", prob)

# Prob[0] Adjustment
prob[0] = torch.max(prob[0], prob[-1])
prob = prob[:-1]
print("Probabilities After Adjustment:\n", prob)


In [ ]:
import torch
import numpy as np

from transforms3d import quaternions
from lietorch import SE3

In [ ]:
def tq2mat(tq):
    """ Transform translation-quaternion (tq) to (4x4) matrix. """
    tq = np.array(tq)
    T = np.eye(4)
    T[:3, :3] = quaternions.quat2mat(np.roll(tq[3:], 1))
    T[:3, 3] = tq[:3]
    return T


In [ ]:
def lietorch_tq2mat(tq):
    tq = torch.from_numpy(np.array(tq)).view(1, 7)
    T = SE3.InitFromVec(tq)
    return T
    

In [ ]:
tq = [1.1622, 0.6244, 1.4513, 0.6675, 0.6291, -0.2670, -0.2957]

In [ ]:
T = tq2mat(tq)
print(T)

T_lie = lietorch_tq2mat(tq)
print(T_lie.matrix())

print('difference: {}'.format((T - T_lie.matrix().numpy()).sum()))

In [ ]:
import open3d as o3d
import numpy as np

# Initialize Open3D visualization
vis = o3d.visualization.Visualizer()
vis.create_window()

# Initialize point cloud and camera pose (as a 4x4 transformation matrix)
point_cloud = o3d.geometry.PointCloud()
camera_pose = np.eye(4)

# Add initial point cloud and camera pose to the visualizer
vis.add_geometry(point_cloud)

# Set some visualization settings
render_option = vis.get_render_option()
render_option.point_size = 2.0  # example setting, you can customize this

# Define a callback function
def update_visualization(vis):
    global point_cloud, camera_pose
    
    # Simulate updated data from your visual odometry algorithm
    new_points = np.random.rand(100, 3)  # Replace this with the updated point cloud from your algorithm
    new_pose = np.eye(4)  # Replace this with the updated camera pose from your algorithm
    new_pose[0, 3] = np.random.rand()  # Simulate some translation in x-direction
    
    # Update point cloud data
    point_cloud.points = o3d.utility.Vector3dVector(new_points)
    
    # Update camera pose
    camera_pose = np.dot(camera_pose, np.linalg.inv(new_pose))
    
    # Update the camera view
    ctr = vis.get_view_control()
    cam = ctr.convert_to_pinhole_camera_parameters()
    cam.extrinsic = camera_pose
    ctr.convert_from_pinhole_camera_parameters(cam)
    
    return False

# Run the callback function in a loop
for i in range(100):  # Run the loop 100 times
    update_visualization(vis)
    vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()

# Close the visualization
vis.run()
vis.destroy_window()

In [8]:
import numpy as np

acc_ids = [0, 3, 100]
index = 3
print(max(np.searchsorted(acc_ids, index+1) - 1, 0))

1


In [1]:
import numpy as np

a = np.random.rand(200, 300)
print(a.shape, a[np.newaxis, :].shape)

(200, 300) (1, 200, 300)


In [5]:
import torch

n = 5
print(n//2)
neighborhood = torch.meshgrid(
        torch.arange(-(n//2), (n//2)+1).float(),
        torch.arange(-(n//2), (n//2)+1).float()
    )
print(neighborhood)

2
(tensor([[-2., -2., -2., -2., -2.],
        [-1., -1., -1., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.]]), tensor([[-2., -1.,  0.,  1.,  2.],
        [-2., -1.,  0.,  1.,  2.],
        [-2., -1.,  0.,  1.,  2.],
        [-2., -1.,  0.,  1.,  2.],
        [-2., -1.,  0.,  1.,  2.]]))


In [7]:
a = torch.zeros((2, 2))
a = torch.Tensor([])
print(a.nelement())

0


In [11]:
a = torch.rand((3, 1, 12, 16))
h, w = a.shape[2:]
print(h, w)
c = torch.rand((3, 1))[..., None, None]
y, x = torch.meshgrid(
        torch.arange(h),
        torch.arange(w))

x = x - c
y = y - c
p = torch.stack([x, y, a], dim=-1)
print(a.shape)

12 16
torch.Size([3, 1, 12, 16])


In [28]:
from lietorch import SO3, SE3

a = torch.randn(4, 7, device='cuda', requires_grad=True)
print(a)
R = SE3.InitFromVec(a)
print(R.matrix())
print(R.vec())

point = torch.randn(4, 90, 3, device='cuda')
a  = R[:, None].act(point)

tensor([[-1.9168, -2.9213, -0.1774, -0.5586, -0.4666,  0.5055, -2.3070],
        [ 0.6641,  1.2612, -0.6778, -0.6633, -0.2532,  0.1366, -0.0294],
        [ 0.0081, -1.6798, -0.6679, -0.1320, -0.7448,  0.1356, -0.3184],
        [ 1.0681,  1.4154,  1.3340, -1.7683, -1.3022, -0.2429,  0.1500]],
       device='cuda:0', requires_grad=True)
tensor([[[ 0.8450,  0.4672,  0.2600, -1.9168],
         [-0.2965,  0.8141, -0.4993, -2.9213],
         [-0.4450,  0.3448,  0.8265, -0.1774],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.6839,  0.6568, -0.3176,  0.6641],
         [ 0.6261, -0.7518, -0.2066,  1.2612],
         [-0.3745, -0.0576, -0.9254, -0.6778],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[-0.6566,  0.4090,  0.6337,  0.0081],
         [ 0.1593,  0.8965, -0.4135, -1.6798],
         [-0.7372, -0.1706, -0.6538, -0.6679],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.2843,  0.9540,  0.0955,  1.0681],
         [ 0.9242, -0.2992,  0.2371,  1.4154],